In [2]:
import sys
from pathlib import Path

try:
    # 尝试作为脚本运行
    file_path = Path(__file__).resolve()
    # 根据你的原代码，这里是脚本所在目录往上找 3 层
    ROOT_DIR = file_path.parent.parent.parent
except NameError:
    # 如果报错，说明在 Jupyter Notebook 中
    # 注意：Path.cwd() 直接就是目录，不需要去掉文件名那一层，所以 .parent 数量通常比脚本少一个
    # 请务必 print 确认路径是否正确
    ROOT_DIR = Path.cwd().parent.parent 
    
print(f"当前设置的根目录: {ROOT_DIR}")
sys.path.insert(0, str(ROOT_DIR))

from camel.agents import ChatAgent
from dotenv import load_dotenv
from src.settings import settings  # 使用 settings 单例

# 访问配置
print(f"OPENAI_API_KEY: {settings.OPENAI_API_KEY}")

当前设置的根目录: /mnt/ssd2/steins/wenkai/project/doc-reading-agent-demo
OPENAI_API_KEY: EMPTY


In [3]:
from camel.agents import ChatAgent
from camel.models import ModelFactory
from camel.types import ModelPlatformType, ModelType
from src.code.Tools.page_indexing_tool import get_page_index_tool

from camel.agents import ChatAgent
from camel.toolkits import FunctionTool


# Create agent with tool
agent = ChatAgent(tools=[get_page_index_tool()])

while True:
    user_input = input("输入你的问题 (输入 'exit' 或 'quit' 退出): ")

    if user_input.lower() in ["exit", "quit"]:
        print("Exiting the conversation.")
        break
    response = agent.step(user_input)
    print(response.msg.content)

2026-01-08 10:29:54,707 - camel.camel.types.enums - WARNING - Unknown model type Qwen/Qwen3-30B-A3B, set maximum token limit to 999_999_999
Exiting the conversation.


In [4]:
import os
import sys
from camel.agents import ChatAgent
from camel.toolkits import FunctionTool
from camel.models import ModelFactory
from camel.types import ModelPlatformType
from src.code.Tools.page_indexing_tool import get_page_index_tool
from src.code.Tools.visual_reader_tool import get_visual_reader_tool
from loguru import logger

logger = logger.bind(name="POC-Test")
logger.remove()
logger.add(sys.stdout, level="ERROR", format="{time} {level} {message}")

os.environ["DEFAULT_MODEL_TYPE"] = "Qwen/Qwen3-30B-A3B"

# 1. 获取当前工作目录 (你的 Notebook 所在位置)
# 结果: .../doc-reading-agent-demo/src/POC
current_work_dir = os.getcwd()
# 2. 往上退两级，找到项目根目录
project_root = os.path.dirname(os.path.dirname(current_work_dir))
# 3. 拼接正确的绝对路径
abs_file_path = os.path.join(project_root, "示例数据", "test.docx")

# 验证一下
logger.info(f"当前计算出的路径: {abs_file_path}")
if os.path.exists(abs_file_path):
    logger.info("✅ 成功找到文件！")
else:
    logger.info("❌ 还是没找到，请检查层级是否正确。")

sys_msg = f"""你是一个专业的文档阅读助手。
文件地址是：{abs_file_path}与及对应的pdf文件。
当用户询问文档的具体内容或结构时，请务必调用 `parse_structure` 工具。
随后你会定位到具体页码，再调用 `visual_reader` 工具来读取对应页面的内容并回答用户的问题。
请你务必根据工具返回的信息来回答用户的问题，而不是凭空编造信息。
且回答时提供信息来源的具体定位。
"""

llm = ModelFactory.create(
    model_platform=ModelPlatformType.OPENAI_COMPATIBLE_MODEL,
    model_type=settings.LLM_NAME,
    api_key=None,
    model_config_dict={
        "max_tokens": 4096,
    }
)

doc_tool = get_page_index_tool()
image_reader_tool = get_visual_reader_tool()

agent = ChatAgent(
    system_message=sys_msg,
    tools=[doc_tool, image_reader_tool], # 使用包装好的 tool
    message_window_size=10 # 限制记忆窗口，防止上下文过长
)

# 4. 发起对话 (关键：传入绝对路径)
first_prompt = f"请你介绍一下自己，并告诉我如何使用你"

print("-" * 50)
print("User:", first_prompt)
response = agent.step(first_prompt)
print("Agent:", response.msg.content)

# 5. 进入交互循环
while True:
    print("-" * 50)
    user_input = input("输入你的问题 (输入 'exit' 退出): ")

    if user_input.lower() in ["exit", "quit"]:
        break
    
    # 如果用户在后续对话中只说了文件名，你可能需要在这里处理路径逻辑，
    # 但如果是同一个文件，Agent 的上下文记忆通常能处理。
    response = agent.step(user_input)
    print("Agent:", response.msg.content)

2026-01-08 10:30:06,310 - camel.camel.types.enums - WARNING - Unknown model type Qwen/Qwen3-30B-A3B, set maximum token limit to 999_999_999
--------------------------------------------------
User: 请你介绍一下自己，并告诉我如何使用你
2026-01-08 10:30:06,463 - camel.camel.types.enums - WARNING - Unknown model type Qwen/Qwen3-30B-A3B, set maximum token limit to 999_999_999
2026-01-08 10:30:06,465 - camel.camel.types.enums - WARNING - Unknown model type Qwen/Qwen3-30B-A3B, set maximum token limit to 999_999_999
Agent: 你好！我是你的文档阅读助手，专门用于帮助你解析和理解文档内容。我可以读取 Word（.docx）和 PDF 格式的文档，分析其结构，并根据你的问题精准定位到具体页面，提供详细的内容回答。

### 我能为你做什么？
- **解析文档结构**：快速获取文档的目录、章节、页码等信息。
- **精准定位内容**：根据你的问题，定位到具体页码并提取相关内容。
- **多格式支持**：支持 `.docx` 和对应的 `.pdf` 文件。
- **可视化阅读**：结合图像识别技术，准确读取页面上的文字、表格、图表等内容。

---

### 如何使用我？
请按照以下步骤操作：

1. **提供文档路径**  
   告诉我你的 Word 文档和 PDF 文件的路径，例如：
   ```
   Word 文档路径: /mnt/ssd2/steins/wenkai/project/doc-reading-agent-demo/示例数据/test.docx
   PDF 文档路径: /mnt/ssd2/steins/wenkai/project/doc-reading-agent-demo/示例数

In [13]:
agent.tool_dict

{'page_index_tool': <camel.toolkits.function_tool.FunctionTool at 0x740ecc9a4440>,
 'read_page': <camel.toolkits.function_tool.FunctionTool at 0x740ecd6711c0>}

In [1]:
from pymilvus import connections, utility

def test_connect():
    # 替换为你公司的配置
    HOST = '192.168.3.112'
    PORT = '19530' 
    
    print(f"正在连接 Milvus {HOST}:{PORT}...")
    try:
        connections.connect("default", host=HOST, port=PORT)
        print("✅ 连接成功！")
        print(f"现有集合: {utility.list_collections()}")
    except Exception as e:
        print(f"❌ 连接失败: {e}")

if __name__ == "__main__":
    test_connect()

正在连接 Milvus 192.168.3.112:19530...
✅ 连接成功！
现有集合: ['memory_collection', 'jiakai_test_collection', 'default', 'bidding_test', 'image_collection']


In [15]:
def add(a:int, b:int) -> int:
    """此函数用于计算两个数的和

    Args：
        a: 第一个数
        b: 第二个数

    Returns:
        a + b
    """
    return a + b 
add_tool = FunctionTool(add)
print(add_tool.get_openai_function_schema())
print(add_tool.get_openai_tool_schema())
print(add_tool.get_function_description())

{'name': 'add', 'description': '此函数用于计算两个数的和\nArgs：\n    a: 第一个数\n    b: 第二个数', 'strict': True, 'parameters': {'properties': {'a': {'type': 'integer'}, 'b': {'type': 'integer'}}, 'required': ['a', 'b'], 'type': 'object', 'additionalProperties': False}}
{'type': 'function', 'function': {'name': 'add', 'description': '此函数用于计算两个数的和\nArgs：\n    a: 第一个数\n    b: 第二个数', 'strict': True, 'parameters': {'properties': {'a': {'type': 'integer'}, 'b': {'type': 'integer'}}, 'required': ['a', 'b'], 'type': 'object', 'additionalProperties': False}}}
此函数用于计算两个数的和
Args：
    a: 第一个数
    b: 第二个数
